# eQTL chr13 overlapping mQTL chr13

Table of all eQTLs

In [17]:
.libPaths("~/R/x86_64-redhat-linux-gnu-library/3.2/")
# config opts and libraries
options(repr.plot.width = 6)
options(repr.plot.height = 5)
library(ggplot2);
library(plyr);
library(dplyr);
library(reshape2);
library(LSD);
library(pheatmap);
library(parallel);
options(mc.cores = 5);
library(stringr);
library(RColorBrewer);
library(qtl);
library(GenomicRanges);

In [2]:
# load normalized QTLs
load("/g/steinmetz/brooks/genphen//metabolome/qtls/mQTLs_comball_funqtl_2014.rda")
mQTLs_funqtl_2014_class = sapply(mQTLs_funqtl_2014,class)
mQTLs_funqtl_2014 = mQTLs_funqtl_2014[mQTLs_funqtl_2014_class!="try-error"]

In [3]:
# load eQTLs
load("/g/steinmetz/brooks/genphen/transcriptome/qtl/eQTL_08032016.rda")

In [4]:
# load genotype and markers files
genotype_f = "/g/steinmetz/brooks/yeast/genomes/S288CxYJM789/genotypes_S288c_R64.rda"
load(genotype_f)

In [ ]:
co = .1 # 10% FDR
bayesint = .95 # 95% Bayesian confidence interval around QTL

# normalized data
genes = colnames(eQTL$qtls)[c(-1,-2)]
genes_i = seq(3, dim(eQTL$qtls)[2])
qtltable = do.call(rbind,lapply(genes_i, function(i){
    g = genes[i]
    o = try({
        #print(i)
        m = i
        d = data.frame(as.matrix(eQTL$qtls[,1:2]), lod = eQTL$qtls[, i+2])
        chrs = unique(d[d$lod >= summary(eQTL$resamples[[i]], alpha = co)[1],][,"chr"])
        qtl_intervals = list()
        if (length(chrs)>0) {
          for (j in chrs) {
            bi = bayesint(eQTL$qtls, chr = str_pad(j, 2, pad = "0"), prob=bayesint, lodcolumn = i)
            lod = max(bi[,i+2])
            maxmrk = rownames(bi)[which(bi[,i+2]==lod)]
            #print(lod)
            qtl_intervals[[j]] = try(mrk[rownames(bi)],silent = T)
            if (class(qtl_intervals[[j]])=="try-error") {
              qtl_intervals[[j]] = NULL
            } else {
              nn = sapply(as.character(seqnames(qtl_intervals[[j]])),function(x){
                paste(substr(x,1,3),as.roman(substr(x,4,5)),sep="")
              })
              qtl_intervals[[j]] = renameSeqlevels(qtl_intervals[[j]],nn)
              qtl_intervals[[j]] = keepSeqlevels(qtl_intervals[[j]],unique(nn))
              qtl_intervals[[j]] = range(qtl_intervals[[j]])
              qtl_intervals[[j]]$maxlod = lod
              qtl_intervals[[j]]$maxmrk = maxmrk
              qtl_intervals[[j]] = as.data.frame(qtl_intervals[[j]])
            }
            #print(qtl_intervals)
          }
        }
        if (length(qtl_intervals) > 1) {
            qtl_df = do.call(rbind,qtl_intervals)
        } else {
            qtl_df = as.data.frame(qtl_intervals[[1]])
        }
        qtl_df = cbind(gene = g, qtl_df)
        })
        if (class(o)!="try-error") {
            return(o)
        } else {
            return(NULL)
        }
}))
# load eQTLs
save(eQTL, qtltable,file = "/g/steinmetz/brooks/genphen/transcriptome/qtl/eQTL_08032016.rda")

In [14]:
cat("Number of eQTLs:", dim(qtltable)[1], "@ FDR:", co)

Number of eQTLs: 4150 @ FDR: 0.1

In [16]:
cat("Number of transcripts with at least 1 eQTL:", length(levels(qtltable$gene)), "@ FDR:", co)

Number of transcripts with at least 1 eQTL: 3350 @ FDR: 0.1